In [1]:
import numpy as np
import pandas as pd

# Preprocessing

In [2]:
COLUMNS = ['station','date','feature', 'value', 'measurement','quality', 'source', 'hour']

In [3]:
#Transform Data to meaningful features
df = pd.read_csv('../data/2014.csv', header=None, names=COLUMNS)

In [4]:
df = df.append(pd.read_csv('../data/2015.csv', header=None, names=COLUMNS))
df = df.append(pd.read_csv('../data/2016.csv', header=None, names=COLUMNS))
df = df.append(pd.read_csv('../data/2017.csv', header=None, names=COLUMNS))

In [5]:
df.head()

,station,date,feature,value,measurement,quality,source,hour
0,ASN00015643,20140101,TMAX,424,NaN,NaN,a,NaN
1,ASN00015643,20140101,TMIN,227,NaN,NaN,a,NaN
2,ASN00015643,20140101,PRCP,0,NaN,NaN,a,NaN
3,US1MNCV0008,20140101,PRCP,0,NaN,NaN,N,NaN
4,US1MNCV0008,20140101,SNOW,0,NaN,NaN,N,NaN


In [24]:
#Only selecting features who are available in Test Set + Feature to be predicted
selected_features = ['TMIN']

In [25]:
df = df[df['feature'].isin(selected_features)]

In [26]:
df_pivot = df.pivot_table(index=['station','date'], columns='feature', values='value', aggfunc=np.min)

In [27]:
#Include Location Information?
df_stations = pd.read_csv('../data/ghcnd-stations.csv', header=None, names=['station','lat', 'long', 'elev'], sep=';')

In [28]:
df_stations = df_stations.set_index('station')

In [58]:
df_time = df_pivot['TMIN']
df_time = df_time.reset_index()
len(df_time)

18683824

In [55]:
stations = df_time.station.unique()

In [59]:
def derive_nth_day_feature(data, feature, N): 
    col_name = "{}_{}".format(feature, N)
    data[col_name] = [None]*len(data)
    #col = data.columns.get_loc(feature)
    for station in stations:
        rows = len(data[data['station']==station])
        index = data[data['station']==station].index
        i = index[0]
        print (station, '\r', end='')
        for r in range(i+N, rows):
            data.at[r,col_name] = data.loc[r-N,feature]
            #print (r, '\r', end='')
    return data

In [ ]:
for N in range(1, 5):
    df_train = derive_nth_day_feature(df_time, 'TMIN', N)
    print (N)
            
df_train.head()

In [ ]:
for N in range(5, 21):
    df_train = derive_nth_day_feature(df_time, 'TMIN', N)
    print (N)
            
df_train.head()